In [0]:
from databricks.vector_search.client import VectorSearchClient

# Initialize client
vsc = VectorSearchClient()

# Get index
index = vsc.get_index(index_name="docai-dbx.gold.doc_embeddings_index")

# Use the built-in embedding function for querying (same one used for index creation)
results = index.similarity_search(
    query_text="What is the refund policy?",
    columns=["chunk_id", "file_name", "text_chunk"],
    num_results=5
)
display(results)
# Show results
# for r in results['result']:
#     print(f"📄 File: {r['file_name']}")
#     print(f"📝 Chunk: {r['text_chunk'][:300]}...\n---\n")


In [0]:
# if isinstance(results["result"], list):
#     context = "\n\n".join([r["text_chunk"] for r in results["result"]])
# else:
#     context = results["result"]

# query = "What is the refund policy?"

# prompt = f"""Use the following document excerpts to answer the question.

# Document:
# {context}

# Question: {query}
# Answer:"""

# print(prompt)

In [0]:
%pip install openai

In [0]:
 %restart_python

In [0]:
from openai import OpenAI
import os

# Use token from current notebook session
DATABRICKS_TOKEN = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()

# Init client with Databricks endpoint as base_url
client = OpenAI(
    api_key=DATABRICKS_TOKEN,
    base_url="https://dbc-af42bc48-2730.cloud.databricks.com/serving-endpoints"  # replace!
)

# Construct context from vector search
if isinstance(results["result"], list):
    context = "\n\n".join([r["text_chunk"] for r in results["result"]])
else:
    context = results["result"]
query = "what is the name of the applicant?"

# Ask the LLM
chat_completion = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "You are a helpful assistant. Answer questions based on the document excerpts provided."},
        {"role": "user", "content": f"""Use the following document to answer the question.

Document:
{context}

Question: {query}
"""}
    ],
    model="databricks-llama-4-maverick",  # or another model you deployed
    max_tokens=300
)

# Print answer
print(chat_completion.choices[0].message.content)